# Recommender system

## 1. Load data from DB

In [ ]:
from model.warehouse import UserInteractions
from sqlalchemy.orm import sessionmaker
from lightfm.data import Dataset
from config.warehouse_config import get_db_engine

Session = sessionmaker(bind=get_db_engine())
session = Session()
dataset = Dataset()

user_interactions = session.query(UserInteractions).all()

In [ ]:
user_ids = [interaction.user_id for interaction in user_interactions]
item_ids = [interaction.title_id for interaction in user_interactions]
ratings = [interaction.rating for interaction in user_interactions]

dataset.fit(users=user_ids, items=item_ids)

interactions, weights = dataset.build_interactions(zip(user_ids, item_ids, ratings))

## 2. Preprocess data & train model

In [ ]:
from model.warehouse import Genres
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

genres = session.query(Genres).all()
genre_id_map = {genre.id: index + 1 for index, genre in enumerate(genres)}

genre_ids_adjusted = [genre_id_map[genre.id] for genre in genres]
dataset.fit_partial(items=genre_ids_adjusted, item_features=genre_ids_adjusted)



In [ ]:
item_features = dataset.build_item_features([(genre_id_map[genre.id], [genre.id]) for genre in genres])

In [ ]:
model = LightFM(loss='warp')
model.fit(interactions, item_features=item_features, epochs=20)

## 3. Evaluate model

In [ ]:
train_precision = precision_at_k(model, interactions, item_features=item_features, k=10).mean()
print(f'Train precision: {train_precision}')